In [12]:
pip install requests


[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
pip install pandas


[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [14]:
import requests
import pandas as pd
from datetime import datetime

# API endpoint and parameters
WEATHER_ENDPOINT = "https://api.openweathermap.org/data/2.5/weather"
API_KEY = "5364c78eab167c6b826b39227030cfa4"  # Replace with your actual API key


provinces = {
    "Chiang Mai":{
        "lat": 14.0134,
        "lon": 98.6601
    },
    "Lamphun":{
        "lat": 18.5745,
        "lon": 99.0087
    },
    "Lampang":{
        "lat": 17.6289,
        "lon": 100.0977
    },
    "Uttaradit":{
        "lat": 17.6255,
        "lon": 100.0942
    },
    "Phrae":{
        "lat": 18.1459,
        "lon": 100.1410
    },
    "Nan":{
        "lat": 18.7838,
        "lon": 100.7789
    },
    "Phayao":{
        "lat": 19.1920,
        "lon": 99.8788
    },
    "Chiang Rai":{
        "lat": 19.9086 ,
        "lon": 99.8325
    },
    "Mae Hong Son":{
        "lat": 19.3006 ,
        "lon": 97.9686
    }
}

# Function to fetch and process weather data
def get_weather_data(province='Pathum Thani'):
    
    params = {
        "lat": provinces[province]['lat'],
        "lon": provinces[province]['lon'],
        "appid": API_KEY,
        "units": "metric"
    }
    try:
        # Make API request
        response = requests.get(WEATHER_ENDPOINT, params=params)
        response.raise_for_status()  # Raise an exception for bad status codes
        data = response.json()
        
        # Convert timestamp to datetime
        timestamp = datetime.fromtimestamp(data['dt'])
        
        # Create dictionary with required fields
        weather_dict = {
            'timestamp': timestamp,
            'year': timestamp.year,
            'month': timestamp.month,
            'day': timestamp.day,
            'hour': timestamp.hour,
            'minute': timestamp.minute,
            'requested_province':province,
            'location': data['name'],
            'weather_main': data['weather'][0]['main'],
            'weather_description': data['weather'][0]['description'],
            'main.temp': data['main']['temp']
        }
        
        # Create DataFrame
        # df = pd.DataFrame([weather_dict])
        
        # return df
        return weather_dict
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None
    except KeyError as e:
        print(f"Error processing data: Missing key {e}")
        return None

In [15]:
df=pd.DataFrame([get_weather_data(p) for p in list(provinces.keys())])
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   timestamp            9 non-null      datetime64[ns]
 1   year                 9 non-null      int64         
 2   month                9 non-null      int64         
 3   day                  9 non-null      int64         
 4   hour                 9 non-null      int64         
 5   minute               9 non-null      int64         
 6   requested_province   9 non-null      object        
 7   location             9 non-null      object        
 8   weather_main         9 non-null      object        
 9   weather_description  9 non-null      object        
 10  main.temp            9 non-null      float64       
dtypes: datetime64[ns](1), float64(1), int64(5), object(4)
memory usage: 920.0+ bytes


,timestamp,year,month,day,hour,minute,requested_province,location,weather_main,weather_description,main.temp
0,2025-04-28 17:30:04,2025,4,28,17,30,Chiang Mai,Dawei,Clouds,overcast clouds,24.56
1,2025-04-28 17:30:04,2025,4,28,17,30,Lamphun,Lamphun,Clouds,broken clouds,35.21
2,2025-04-28 17:30:04,2025,4,28,17,30,Lampang,Ban Tha It Bon,Clouds,broken clouds,33.01
3,2025-04-28 17:30:04,2025,4,28,17,30,Uttaradit,Uttaradit,Clouds,broken clouds,33.01
4,2025-04-28 17:27:03,2025,4,28,17,27,Phrae,Phrae,Rain,light rain,33.36


In [16]:
import pandas as pd

# lakeFS credentials from your docker-compose.yml
ACCESS_KEY = "access_key"
SECRET_KEY = "secret_key"

# lakeFS endpoint (running locally)
#lakefs_endpoint = "http://lakefs-dev:8000/"
lakefs_endpoint = "http://localhost:8000/"

# lakeFS repository, branch, and file path
repo = "weather"
branch = "main"
path = "weather.parquet"

# Construct the full lakeFS S3-compatible path
lakefs_s3_path = f"s3a://{repo}/{branch}/{path}"

# Configure storage_options for lakeFS (S3-compatible)
storage_options = {
    "key": ACCESS_KEY,
    "secret": SECRET_KEY,
    "client_kwargs": {
        "endpoint_url": lakefs_endpoint
    }
}

In [19]:
pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.9/30.9 MB 8.4 MB/s eta 0:00:0000:0100:01m

[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [20]:
pip install fastparquet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.8/683.8 kB 6.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.7 MB/s eta 0:00:00a 0:00:01
  Using cached fsspec-2025.3.2-py3-none-any.whl (194 kB)

[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [22]:
pip install s3fs

  Using cached s3fs-2025.3.2-py3-none-any.whl (30 kB)
  Using cached aiobotocore-2.21.1-py3-none-any.whl (78 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.8/457.8 kB 7.1 MB/s eta 0:00:00a 0:00:01
  Using cached aioitertools-0.12.0-py3-none-any.whl (24 kB)
  Using cached botocore-1.37.1-py3-none-any.whl (13.4 MB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl (15 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.2/122.2 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 4.5 MB/s eta 0:00:00

[notice] A new release of pip available: 22.3 -> 25.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour']
)

df.to_parquet(
    lakefs_s3_path,
    storage_options=storage_options,
    partition_cols=['year','month','day','hour']
)

In [24]:
from datetime import datetime
import pytz

dt = datetime.now()
thai_tz = pytz.timezone('Asia/Bangkok')
dt = dt.replace(tzinfo=thai_tz)